### 识别表格图片，对表格里的字段数据准确提取，保存为JSON格式！
使用OCR识别，利用大模型生成结果。
调用moonshot模型

In [ ]:
import json
import re
from pathlib import Path
from openai import OpenAI
 
client = OpenAI(
    api_key="", 
    base_url="https://api.moonshot.cn/v1",
)

图片识别

In [2]:
file_object = client.files.create(file=Path("19.jpg"), purpose="file-extract")

file_content = client.files.content(file_id=file_object.id).text
 

file_content长度可能太长，进行去重。

In [3]:
 
text =  file_content 

# 分块
blocks = re.split(r"\| \| \| \| \| \| \|", text)

# 每个块进行去重
def remove_duplicates(block):
    seen = set()
    unique_elements = []
    for part in block.split(" | "):
        if part not in seen:
            unique_elements.append(part)
            seen.add(part)
    return " | ".join(unique_elements)


deduped_blocks = [remove_duplicates(block) for block in blocks]

# 拼接在一起
deduped_text = " | | | | | | | ".join(deduped_blocks)

print(deduped_text)

{"content":"### 中华人民共和国海关出口货物报关单\n\n预录入编号:\n\n海关编号:\n\n\n\n| 中华人民共和国海关出口货物报关单 预录入编号: 海关编号: | 中华人民共和国海关出口货物报关单 预录入编号: 海关编号: | 中华人民共和国海关出口货物报关单 预录入编号: 海关编号: |\n| --- | --- | --- |\n| 收发货人(9144060007185672XW) 日丰企业(佛山)有限公司 | 收发货人(9144060007185672XW) 日丰企业(佛山)有限公司 | 出口口岸 YANTIAN | 出口日期 申报日期 2024.09.25 2024.09.25 | 出口日期 申报日期 2024.09.25 2024.09.25 |\n| 生产销售单位 日丰企业(佛山)有限公司 | 生产销售单位 日丰企业(佛山)有限公司 | 运输方式 海运 | 运输工具名称 |  | 提运单号 |  |\n| 申报单位 | 申报单位 | 监管方式 一般贸易 | 征免性质 一般征税 | 备案号 | 备案号 |\n| 贸易国(地区) 瑞典 | 贸易国(地区) 瑞典 | 运抵国(地区) 瑞典 | 指运港 Gothenburg | 境内货源地 佛山其他 | 境内货源地 佛山其他 |\n| 许可证号 | 许可证号 | 成交方式 FOB | 运费 0.00 | 保费 0.00 | 杂费 0.00 | 杂费 0.00 |\n| 合同协议 SWE022430/31/32/24/28B | 合同协议 SWE022430/31/32/24/28B | 件数 2708 | 包装种类 纸箱 | 毛重(千克) 净重(千克) 28063.87 21091.98 | 毛重(千克) 净重(千克) 28063.87 21091.98 |\n| 集装箱号 随附单据 生产厂家 40HQ*4柜 日丰企业(佛山)有限公司 标记唛码及备注 N/M | 集装箱号 随附单据 生产厂家 40HQ*4柜 日丰企业(佛山)有限公司 标记唛码及备注 N/M | 集装箱号 随附单据 生产厂家 40HQ*4柜 日丰企业(佛山)有限公司 标记唛码及备注 N/M |\n| 结汇/退税 | 结汇/退税 | 结汇/退税 |\n| 项目 | 商品编号 | 商品名称、规格型号 | 数量 | 单位 | 终目的

构建message，获取回答

In [4]:



# 把文件内容通过系统提示词 system prompt 放进请求中
prompt = """您好，我需要您将以下图片内容识别并整理成表格形式的JSON数据。请确保每个识别到的元素都被正确分类，并按照表格的逻辑结构化数据。以下是图片内容识别和表格化的要求：

1. 识别图片中的文本和数据：请仔细查看图片，识别出所有的文本和数据信息。
2. 分类信息：将识别到的信息按照逻辑分类，比如标题、日期、项目、数值等。
3. 创建表格结构：根据识别到的信息，创建一个表格结构，每个类别作为表格的一列。
4. 填充数据：将识别到的数据填充到对应的表格列中。
5. 输出JSON格式：将表格数据转换为JSON格式，确保每个字段和值都被正确地表示。
6. 确保图片中的所有信息都在回复中。
只返回json数据内容！
"""
messages = [
    # {
    #     "role": "system",
    #     "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。",
    # },
    {
        "role": "system",
        "content": deduped_text, # <-- 这里，我们将抽取后的文件内容（注意是文件内容，而不是文件 ID）放置在请求中
    },
    {"role": "user", "content": prompt},
]
 
# 然后调用 chat-completion, 获取 Kimi 的回答
completion = client.chat.completions.create(
  model="moonshot-v1-32k",
  messages=messages,
  temperature=0, 
  max_tokens=4000,
)
print(completion.choices[0].message)


ChatCompletionMessage(content='{\n  "报关单信息": {\n    "预录入编号": "",\n    "海关编号": "",\n    "收发货人": "日丰企业(佛山)有限公司",\n    "统一社会信用代码": "9144060007185672XW",\n    "出口口岸": "YANTIAN",\n    "出口日期": "2024.09.25",\n    "申报日期": "2024.09.25",\n    "生产销售单位": "日丰企业(佛山)有限公司",\n    "运输方式": "海运",\n    "监管方式": "一般贸易",\n    "征免性质": "一般征税",\n    "贸易国(地区)": "瑞典",\n    "运抵国(地区)": "瑞典",\n    "指运港": "Gothenburg",\n    "境内货源地": "佛山其他",\n    "成交方式": "FOB",\n    "运费": "0.00",\n    "保费": "0.00",\n    "杂费": "0.00",\n    "合同协议": "SWE022430/31/32/24/28B",\n    "件数": "2708",\n    "包装种类": "纸箱",\n    "毛重(千克)": "28063.87",\n    "净重(千克)": "21091.98",\n    "集装箱号": "40HQ*4柜",\n    "生产厂家": "日丰企业(佛山)有限公司",\n    "标记唛码及备注": "N/M"\n  },\n  "商品信息": [\n    {\n      "项目": "1",\n      "商品编号": "3917390000",\n      "商品名称": "铝塑复合管",\n      "规格型号": "LK牌,国外其他牌子,无中文品牌|用于室内供水采暖系统|铝28%、塑料72%|与铝合制|没有附件|厂家:日丰企业(佛山)有限公司|最小爆破压力: 4.0MPA|规格型号:PEX-AL-PEX",\n      "数量": "5532.5",\n      "单位": "千克",\n      "终目的国(地区)": "瑞典",\n      "单价": "5.8196",\n   

保存为json文件

In [6]:
data = completion.choices[0].message.content
# 将JSON字符串解析为Python字典
data_dict = json.loads(data)
with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(data_dict, f, ensure_ascii=False, indent=4)

print("JSON数据已保存到data.json文件中。")


JSON数据已保存到data.json文件中。


In [7]:

from pprint import pprint

# 读取JSON文件
with open('output.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 使用pprint美化打印输出
pprint(data)

{'商品信息': [{'单价': '5.8196',
           '单位': '千克',
           '商品名称': '铝塑复合管',
           '商品编号': '3917390000',
           '币制': '美元',
           '征免': '照章征税',
           '总价': '32197.13',
           '数量': '5532.5',
           '终目的国(地区)': '瑞典',
           '规格型号': 'LK牌,国外其他牌子,无中文品牌|用于室内供水采暖系统|铝28%、塑料72%|与铝合制|没有附件|厂家:日丰企业(佛山)有限公司|最小爆破压力: '
                   '4.0MPA|规格型号:PEX-AL-PEX',
           '项目': '1'},
          {'单价': '20.7042',
           '单位': '千克',
           '商品名称': '不锈钢分水器',
           '商品编号': '8481809000',
           '币制': '美元',
           '征免': '照章征税',
           '总价': '87438.06',
           '数量': '4223.2',
           '终目的国(地区)': '瑞典',
           '规格型号': 'LK牌|型号FN22;|厂家:日丰企业(佛山)有限公司|在输水管道中分导水流|两位n通',
           '项目': '2'},
          {'单价': '15.7895',
           '单位': '千克',
           '商品名称': '密封圈',
           '商品编号': '4016931000',
           '币制': '美元',
           '征免': '照章征税',
           '总价': '3',
           '数量': '0.19',
           '终目的国(地区)': '瑞典',
           '规格型号': '无牌|密

### 题目二：如何在已有LoRA模型上继续训练？LORA应该作用于Transformer的哪个参数矩阵？Rank 如何选取？ 
请详细写自己的思路！
 
<p style="color: #FFA07A;">
如何在已有LoRA模型上继续训练？
</p>
<p style="color: #ADD8E6;">
答：在我看来继续训练已有的 LoRA 模型只需要在已有的LoRA模型基础上继续训练就行。前提是已有的 LoRA 模型已经和base模型合并，并且在继续训练的时候加入一些之前的训练数据，确保能保留之前的知识和能力。
</p>
<p style="color: #FFA07A;">
LORA应该作用于Transformer的哪个参数矩阵？
</p>
<p style="color: #ADD8E6;">
答：作用于Self-Attention的权重矩阵中，其中主要应用于W_q和W_k，当然，之前看过一篇论文，作用于W_q, W_k, W_v, W_o四者会貌似会达到最好效果。
</p>
<p style="color: #FFA07A;">
Rank 如何选取？
</p>
<p style="color: #ADD8E6;">
答：在我个人的工作中，选取的Rank值通常为8或者64，当然，选取其他值也行。对于复杂一点的任务，选取高Rank,反之则低Rank。
</p>